## Using Cloud TPU and a Local runtime in Colab

Make preparations according to the following links:

* Creating a Cloud TPU https://cloud.google.com/tpu/docs/managing-tpus-tpu-vm#tpu-vms

* Preparing Jupyter and connect to a Local runtime https://research.google.com/colaboratory/local-runtimes.html


In [ ]:
!pip install 'jax[tpu]>=0.2.16' -f https://storage.googleapis.com/jax-releases/libtpu_releases.html

Looking in links: https://storage.googleapis.com/jax-releases/libtpu_releases.html
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 1.2 MB 4.4 MB/s 
     |████████████████████████████████| 17.3 MB 68.6 MB/s 
     |████████████████████████████████| 65 kB 6.4 MB/s 
     |████████████████████████████████| 34.5 MB 89.0 MB/s 
     |████████████████████████████████| 72.2 MB 153 kB/s 
     |████████████████████████████████| 170.3 MB 54 kB/s 
  Created wheel for jax: filename=jax-0.4.3-py3-none-any.whl size=1384891 sha256=281f231ccdf58a75c935dbf8bb7a2ffe976396701b549dfa4060a131f722f4aa
  Stored in directory: /

Needed for pretty sharding visualization

In [ ]:
!pip install rich

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 239 kB 4.9 MB/s 
     |████████████████████████████████| 84 kB 4.8 MB/s 


In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
jax.local_devices()

[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
 TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
 TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
 TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
 TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
 TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1),
 TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
 TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]

In [ ]:
jax.__version__

'0.4.3'

## Using Tensor Sharding

In [ ]:
from jax.experimental import mesh_utils
from jax.sharding import PositionalSharding, NamedSharding

In [ ]:
from jax import random

In [ ]:
import numpy as np

### Dot example

In [ ]:
def dot(v1, v2):
  return jnp.vdot(v1, v2)

In [ ]:
rng_key = random.PRNGKey(42)

In [ ]:
vs = random.normal(rng_key, shape=(2_000_000,100))
v1s = vs[:1_000_000,:]
v2s = vs[1_000_000:,:]

v1s.shape, v2s.shape

((1000000, 100), (1000000, 100))

In [ ]:
jax.debug.visualize_array_sharding(v1s)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

### Positional sharding

In [ ]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((8,1)))

In [ ]:
sharding

PositionalSharding([[{TPU 0}]
                    [{TPU 1}]
                    [{TPU 2}]
                    [{TPU 3}]
                    [{TPU 6}]
                    [{TPU 7}]
                    [{TPU 4}]
                    [{TPU 5}]])

In [ ]:
v1sp = jax.device_put(v1s, sharding)

In [ ]:
type(v1sp)

jaxlib.xla_extension.Array

In [ ]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
└───────┘

In [ ]:
v2sp = jax.device_put(v2s, sharding)

In [ ]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
└───────┘

Input is sharded across all the devices.

In [ ]:
d = jax.vmap(dot)(v1sp, v2sp)

In [ ]:
d.shape

(1000000,)

In [ ]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │ TPU 6 │ TPU 7 │ TPU 4 │ TPU 5 │
└───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

In [ ]:
%timeit jax.vmap(dot)(v1sp, v2sp).block_until_ready()

1.71 ms ± 64.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit jax.vmap(dot)(v1s, v2s).block_until_ready()

4.12 ms ± 24.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
jax.make_jaxpr(jax.vmap(dot))(v1s, v2s)

{ lambda ; a:f32[1000000,100] b:f32[1000000,100]. let
    c:f32[1000000] = dot_general[dimension_numbers=(([1], [1]), ([0], [0]))] a b
  in (c,) }

In [ ]:
modules = jax.jit(jax.vmap(dot)).lower(v1s, v2s).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[1000000,100]{0,1:T(8,128)},f32[1000000,100]{0,1:T(8,128)})->f32[1000000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[1000000,100], param_1.2: f32[1000000,100]) -> f32[1000000] {
  %param_0.2 = f32[1000000,100]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[1000000,100]{0,1:T(8,128)} parameter(1)
  %multiply.1 = f32[1000000,100]{0,1:T(8,128)} multiply(f32[1000000,100]{0,1:T(8,128)} %param_0.2, f32[1000000,100]{0,1:T(8,128)} %param_1.2)
  %constant.1 = f32[]{:T(256)} constant(0)
  ROOT %reduce.1 = f32[1000000]{0:T(1024)} reduce(f32[1000000,100]{0,1:T(8,128)} %multiply.1, f32[]{:T(256)} %constant.1), dimensions={1}, to_

In [ ]:
jax.make_jaxpr(jax.vmap(dot))(v1sp, v2sp)

{ lambda ; a:f32[1000000,100] b:f32[1000000,100]. let
    c:f32[1000000] = dot_general[dimension_numbers=(([1], [1]), ([0], [0]))] a b
  in (c,) }

In [ ]:
modules = jax.jit(jax.vmap(dot)).lower(v1sp, v2sp).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[125000,100]{0,1:T(8,128)},f32[125000,100]{0,1:T(8,128)})->f32[125000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[125000,100], param_1.2: f32[125000,100]) -> f32[125000] {
  %param_0.2 = f32[125000,100]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[125000,100]{0,1:T(8,128)} parameter(1)
  %multiply.2 = f32[125000,100]{0,1:T(8,128)} multiply(f32[125000,100]{0,1:T(8,128)} %param_0.2, f32[125000,100]{0,1:T(8,128)} %param_1.2)
  %constant.2 = f32[]{:T(256)} constant(0)
  ROOT %reduce.2 = f32[125000]{0:T(1024)} reduce(f32[125000,100]{0,1:T(8,128)} %multiply.2, f32[]{:T(256)} %constant.2), dimensions={1}, to_apply=%scalar

In [ ]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

In [ ]:
v1sp = jax.device_put(v1s, sharding)
v2sp = jax.device_put(v2s, sharding)

In [ ]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [ ]:
d = jax.vmap(dot)(v1sp, v2sp)
d.shape

(1000000,)

In [ ]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┐
│TPU 0,1│TPU 2,3│TPU 6,7│TPU 4,5│
└───────┴───────┴───────┴───────┘

In [ ]:
%timeit jax.vmap(dot)(v1sp, v2sp).block_until_ready()

1.9 ms ± 32.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit jax.vmap(dot)(v1s, v2s).block_until_ready()

4.07 ms ± 23.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Looking at HLO (note the all-reduce operation)

In [ ]:
modules = jax.jit(jax.vmap(dot)).lower(v1sp, v2sp).compile().compiler_ir()
for hlo in modules:
  print(hlo.to_string())

HloModule jit_dot, is_scheduled=true, entry_computation_layout={(f32[250000,50]{0,1:T(8,128)},f32[250000,50]{0,1:T(8,128)})->f32[250000]{0:T(1024)}}, allow_spmd_sharding_propagation_to_output={true}

%scalar_add_computation (scalar_lhs: f32[], scalar_rhs: f32[]) -> f32[] {
  %scalar_rhs = f32[]{:T(256)} parameter(1)
  %scalar_lhs = f32[]{:T(256)} parameter(0)
  ROOT %add = f32[]{:T(256)} add(f32[]{:T(256)} %scalar_lhs, f32[]{:T(256)} %scalar_rhs)
}

%fused_computation (param_0.2: f32[250000,50], param_1.2: f32[250000,50]) -> f32[250000] {
  %param_0.2 = f32[250000,50]{0,1:T(8,128)} parameter(0)
  %param_1.2 = f32[250000,50]{0,1:T(8,128)} parameter(1)
  %multiply.2 = f32[250000,50]{0,1:T(8,128)} multiply(f32[250000,50]{0,1:T(8,128)} %param_0.2, f32[250000,50]{0,1:T(8,128)} %param_1.2)
  %constant.2 = f32[]{:T(256)} constant(0)
  ROOT %reduce.2 = f32[250000]{0:T(1024)} reduce(f32[250000,50]{0,1:T(8,128)} %multiply.2, f32[]{:T(256)} %constant.2), dimensions={1}, to_apply=%scalar_add_compu

### Using replication

In [ ]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

v1sp = jax.device_put(v1s, sharding.replicate(axis=1))

jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│TPU 0,1│
├───────┤
│TPU 2,3│
├───────┤
│TPU 6,7│
├───────┤
│TPU 4,5│
└───────┘

In [ ]:
A = random.normal(rng_key, shape=(10000,2000))
B = random.normal(rng_key, shape=(2000,5000))

In [ ]:
Ad = jax.device_put(A, sharding.reshape(4, 2).replicate(1))
Bd = jax.device_put(B, sharding.reshape(4, 2).replicate(0))

In [ ]:
jax.debug.visualize_array_sharding(Ad)
jax.debug.visualize_array_sharding(Bd)

┌───────┐
│TPU 0,1│
├───────┤
│TPU 2,3│
├───────┤
│TPU 6,7│
├───────┤
│TPU 4,5│
└───────┘

┌─────────────────────────────┬─────────────────────────────┐
│                             │                             │
│                             │                             │
│                             │                             │
│                             │                             │
│         TPU 0,2,4,6         │         TPU 1,3,5,7         │
│                             │                             │
│                             │                             │
│                             │                             │
│                             │                             │
└─────────────────────────────┴─────────────────────────────┘

In [ ]:
Cd = jnp.dot(Ad, Bd)

In [ ]:
jax.debug.visualize_array_sharding(Cd)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [ ]:
C = A @ B

In [ ]:
jax.numpy.array_equal(C,Cd)

Array(True, dtype=bool)

In [ ]:
jax.debug.visualize_array_sharding(C)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [ ]:
C.shape, Cd.shape

((10000, 5000), (10000, 5000))

In [ ]:
C[12,3], Cd[12,3]

(Array(43.027634, dtype=float32), Array(43.027634, dtype=float32))

In [ ]:
Ca = jnp.dot(A, B)

In [ ]:
jax.numpy.array_equal(C,Ca)

Array(True, dtype=bool)

In [ ]:
jax.debug.visualize_array_sharding(Ca)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [ ]:
d = (Cd - C)

In [ ]:
jnp.max(d), jnp.sum(d)

(Array(0., dtype=float32), Array(0., dtype=float32))

In [ ]:
%timeit jnp.dot(Ad, Bd).block_until_ready()

2.61 ms ± 53.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit jnp.dot(A, B).block_until_ready()

10.6 ms ± 40.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%timeit (A@B).block_until_ready()

10.6 ms ± 40.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Using sharding constraints

In [ ]:
from jax import jit

@jit
def distributed_mul(a, b):
  ad = jax.lax.with_sharding_constraint(a, sharding.replicate(1))
  bd = jax.lax.with_sharding_constraint(b, sharding.replicate(0))
  return jnp.dot(ad, bd)

In [ ]:
sharding = PositionalSharding(mesh_utils.create_device_mesh((4,2)))

In [ ]:
jax.debug.visualize_array_sharding(A)
jax.debug.visualize_array_sharding(B)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

┌────────────────────────────────────────────────────────────┐
│                                                            │
│                                                            │
│                                                            │
│                                                            │
│                           TPU 0                            │
│                                                            │
│                                                            │
│                                                            │
│                                                            │
└────────────────────────────────────────────────────────────┘

In [ ]:
d = distributed_mul(A, B)

In [ ]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [ ]:
d.shape

(10000, 5000)

In [ ]:
@jit
def nondistributed_mul(a, b):
  return jnp.dot(a, b)

In [ ]:
dn = nondistributed_mul(A, B)

In [ ]:
jax.debug.visualize_array_sharding(dn)

┌──────────┐
│          │
│          │
│          │
│          │
│  TPU 0   │
│          │
│          │
│          │
│          │
└──────────┘

In [ ]:
%timeit distributed_mul(A, B).block_until_ready()

22 ms ± 99.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit nondistributed_mul(A, B).block_until_ready()

10.6 ms ± 16 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Named sharding

In [ ]:
from jax.sharding import Mesh
from jax.sharding import PartitionSpec as P
from jax.sharding import NamedSharding

In [ ]:
mesh = Mesh(mesh_utils.create_device_mesh((4,2)), axis_names=('batch', 'features'))
sharding = NamedSharding(mesh, P('batch', 'features'))

In [ ]:
v1sp = jax.device_put(v1s, sharding)
v2sp = jax.device_put(v2s, sharding)

jax.debug.visualize_array_sharding(v1sp)

┌───────┬───────┐
│ TPU 0 │ TPU 1 │
├───────┼───────┤
│ TPU 2 │ TPU 3 │
├───────┼───────┤
│ TPU 6 │ TPU 7 │
├───────┼───────┤
│ TPU 4 │ TPU 5 │
└───────┴───────┘

In [ ]:
d = jax.vmap(dot)(v1sp, v2sp)
d.shape

(1000000,)

### Device placement policy and errors

Different devices:

In [ ]:
sharding_a = PositionalSharding(np.array(jax.devices()[:4]).reshape(4,1))
sharding_b = PositionalSharding(np.array(jax.devices()[4:]).reshape(4,1))

In [ ]:
sharding_a

PositionalSharding([[{TPU 0}]
                    [{TPU 1}]
                    [{TPU 2}]
                    [{TPU 3}]])

In [ ]:
sharding_b

PositionalSharding([[{TPU 4}]
                    [{TPU 5}]
                    [{TPU 6}]
                    [{TPU 7}]])

In [ ]:
v1sp = jax.device_put(v1s, sharding_a)
v2sp = jax.device_put(v2s, sharding_b)

In [ ]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
└───────┘

In [ ]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 4 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
└───────┘

In [ ]:
d = jax.vmap(dot)(v1sp, v2sp)

ValueError: ignored

Different order:

In [ ]:
sharding_a = PositionalSharding(np.array(jax.devices()).reshape(8,1))
sharding_b = PositionalSharding(np.array(jax.devices()[::-1]).reshape(8,1))

In [ ]:
sharding_a, sharding_b

(PositionalSharding([[{TPU 0}]
                     [{TPU 1}]
                     [{TPU 2}]
                     [{TPU 3}]
                     [{TPU 4}]
                     [{TPU 5}]
                     [{TPU 6}]
                     [{TPU 7}]]),
 PositionalSharding([[{TPU 7}]
                     [{TPU 6}]
                     [{TPU 5}]
                     [{TPU 4}]
                     [{TPU 3}]
                     [{TPU 2}]
                     [{TPU 1}]
                     [{TPU 0}]]))

In [ ]:
v1sp = jax.device_put(v1s, sharding_a)
v2sp = jax.device_put(v2s, sharding_b)

In [ ]:
jax.debug.visualize_array_sharding(v1sp)

┌───────┐
│ TPU 0 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 7 │
└───────┘

In [ ]:
jax.debug.visualize_array_sharding(v2sp)

┌───────┐
│ TPU 7 │
├───────┤
│ TPU 6 │
├───────┤
│ TPU 5 │
├───────┤
│ TPU 4 │
├───────┤
│ TPU 3 │
├───────┤
│ TPU 2 │
├───────┤
│ TPU 1 │
├───────┤
│ TPU 0 │
└───────┘

In [ ]:
d = jax.vmap(dot)(v1sp, v2sp)

ValueError: ignored

In [ ]:
d = jax.vmap(dot)(v1sp, v2s)

In [ ]:
jax.debug.visualize_array_sharding(d)

┌───────┬───────┬───────┬───────┬───────┬───────┬───────┬───────┐
│ TPU 0 │ TPU 1 │ TPU 2 │ TPU 3 │ TPU 4 │ TPU 5 │ TPU 6 │ TPU 7 │
└───────┴───────┴───────┴───────┴───────┴───────┴───────┴───────┘

## MLP example

### Preparing data

Install these modules if you created a new empty cloud machine 

In [ ]:
!pip install tensorflow

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |█████████████████▍              | 320.6 MB 149.1 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████████▋| 581.6 MB 148.5 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 588.3 MB 7.0 kB/s 
     |████████████████████████████████| 126 kB 105.2 MB/s 
     |████████████████████████████████| 57 kB 6.8 MB/s 
     |████████████████████████████████| 4.8 MB 88.6 MB/s 
     |████████████████████████████████| 4.7 MB 85.3 MB/s 
     |████████████████████████████████| 1.7 MB 95.5 MB/s 
     |████████████████████████████████| 21.5 MB 82.0 MB/s 
     |████████████████████████████████| 1.1 MB 92.9 MB/s 
     |████████████████████████████████| 6.0 MB 86.5 MB/s 
     |████████████████████████████████| 439 kB 104.2 MB/s 
     |████████████████████████████████| 81 kB 13.8 MB/s 
     |████████████████████████████████| 2.4 MB 88.2 MB/s 
     |████████████████████████████████| 177 kB 101.7 MB/s 
     |████████████████████████████████| 93 kB 2.4 MB/s 
     |████████████████████████████████| 4.9 MB 95.3 MB/s 
     |████████████████████████████████| 781 kB 90.1 MB/s 
     |████████████████████████████████| 233 kB 98.9 MB/s 


In [ ]:
!pip install tensorflow_datasets

/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
     |████████████████████████████████| 5.3 MB 4.6 MB/s 
     |████████████████████████████████| 110 kB 104.4 MB/s 
     |████████████████████████████████| 152 kB 106.9 MB/s 
     |████████████████████████████████| 146 kB 92.6 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 78 kB 10.0 MB/s 
     |████████████████████████████████| 223 kB 88.6 MB/s 
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21485 sha256=add7681dc8d69a7ec9f425444ae41f514636d1a837c91aac4a7c90d9e6166c81
  Stored in directory: /home/grigo/.cache/pip/

In [ ]:
import jax
import tensorflow as tf
import tensorflow_datasets as tfds

data_dir = '/tmp/tfds'

data, info = tfds.load(name="mnist",
                       data_dir=data_dir,
                       as_supervised=True, 
                       with_info=True)

data_train = data['train']
data_test  = data['test']

2023-02-15 14:39:10.586863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-15 14:39:11.233063: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-15 14:39:11.233197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-15 14:39:11.233206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use

Dl Completed...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 26.07 file/s]
2023-02-15 14:39:13.454293: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/lib
2023-02-15 14:39:13.454326: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)


Dataset mnist downloaded and prepared to /tmp/tfds/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
HEIGHT = 28
WIDTH  = 28
CHANNELS = 1
NUM_PIXELS = HEIGHT * WIDTH * CHANNELS 
NUM_LABELS = info.features['label'].num_classes
BATCH_SIZE  = 32 # total 60k samples
NUM_DEVICES = jax.device_count()

In [ ]:
def preprocess(img, label):
  """Resize and preprocess images."""
  return (tf.cast(img, tf.float32)/255.0), label

train_data = tfds.as_numpy(
    data_train.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)
test_data  = tfds.as_numpy(
    data_test.map(preprocess).batch(NUM_DEVICES*BATCH_SIZE).prefetch(1)
)

In [ ]:
len(train_data)

235

### Preparing MLP

In [ ]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, value_and_grad
from jax import random
from jax.nn import swish, logsumexp, one_hot

In [ ]:
LAYER_SIZES = [28*28, 512, 10]
PARAM_SCALE = 0.01

In [ ]:
def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
  """Initialize all layers for a fully-connected neural network with given sizes"""

  def random_layer_params(m, n, key, scale=1e-2):
    """A helper function to randomly initialize weights and biases of a dense layer""" 
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k, scale) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0), scale=PARAM_SCALE)

In [ ]:
def predict(params, image):
  """Function for per-example predictions."""
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = swish(outputs)
  
  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits

batched_predict = vmap(predict, in_axes=(None, 0))

### Loss and update functions

In [ ]:
INIT_LR = 1.0
DECAY_RATE = 0.95
DECAY_STEPS = 5
NUM_EPOCHS  = 20

In [ ]:
def loss(params, images, targets):
  """Categorical cross entropy loss function."""
  logits = batched_predict(params, images)
  log_preds = logits - logsumexp(logits) # logsumexp trick https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
  return -jnp.mean(targets*log_preds)

@jit
def update(params, x, y, epoch_number):
  loss_value, grads = value_and_grad(loss)(params, x, y)
  lr = INIT_LR * DECAY_RATE ** (epoch_number / DECAY_STEPS)
  return [(w - lr * dw, b - lr * db)
          for (w, b), (dw, db) in zip(params, grads)], loss_value

### Training loop

In [ ]:
@jit
def batch_accuracy(params, images, targets):
  images = jnp.reshape(images, (len(images), NUM_PIXELS))
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == targets)

def accuracy(params, data):
  accs = []
  for images, targets in data:
    accs.append(batch_accuracy(params, images, targets))
  return jnp.mean(jnp.array(accs))

! Be careful. This example does not work with Colab TPU because of Colab bug, it only works with Cloud TPU.

https://github.com/google/jax/issues/8300

If you want to use TPU, you need to build your own runtime on Google Cloud Platform (https://cloud.google.com/tpu/docs/jax-pods) and connects to it using Jupyter (https://research.google.com/colaboratory/local-runtimes.html).

#### 8-way data parallelism

In [ ]:
sharding = PositionalSharding(jax.devices()).reshape(8, 1)

In [ ]:
import time

params = init_params
for epoch in range(NUM_EPOCHS):
  start_time = time.time()
  losses = []
  for x, y in train_data:
    x = jnp.reshape(x, (len(x), NUM_PIXELS))
    y = one_hot(y, NUM_LABELS)
    x = jax.device_put(x, sharding)
    y = jax.device_put(y, sharding)
    params = jax.device_put(params, sharding.replicate())
    params, loss_value = update(params, x, y, epoch)
    losses.append(jnp.sum(loss_value))
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_data)
  test_acc = accuracy(params, test_data)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set loss {}".format(jnp.mean(jnp.array(losses))))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 2.33 sec
Training set loss 0.6914460062980652
Training set accuracy 0.8648548126220703
Test set accuracy 0.874804675579071
Epoch 1 in 1.10 sec
Training set loss 0.6259850859642029
Training set accuracy 0.8882258534431458
Test set accuracy 0.893847644329071
Epoch 2 in 1.11 sec
Training set loss 0.6164931058883667
Training set accuracy 0.8978224396705627
Test set accuracy 0.901074230670929
Epoch 3 in 1.11 sec
Training set loss 0.6100647449493408
Training set accuracy 0.906527042388916
Test set accuracy 0.908984363079071
Epoch 4 in 1.09 sec
Training set loss 0.6044379472732544
Training set accuracy 0.9142231941223145
Test set accuracy 0.917285144329071
Epoch 5 in 1.14 sec
Training set loss 0.5995732545852661
Training set accuracy 0.9211103320121765
Test set accuracy 0.9248046875
Epoch 6 in 1.17 sec
Training set loss 0.5955986976623535
Training set accuracy 0.9261746406555176
Test set accuracy 0.929394543170929
Epoch 7 in 1.12 sec
Training set loss 0.5924407839775085
Training se

In [ ]:
jax.debug.visualize_array_sharding(params[0][0])

┌───────────────────────────────────┐
│                                   │
│                                   │
│                                   │
│                                   │
│        TPU 0,1,2,3,4,5,6,7        │
│                                   │
│                                   │
│                                   │
│                                   │
└───────────────────────────────────┘

#### 4-way data parallelism, 2-way tensor parallelism

In [ ]:
sharding = PositionalSharding(jax.devices()).reshape(4, 2)

In [ ]:
LAYER_SIZES = [28*28, 10000, 10000, 10]
PARAM_SCALE = 0.01

def init_network_params(sizes, key=random.PRNGKey(0), scale=1e-2):
  """Initialize all layers for a fully-connected neural network with given sizes"""

  def random_layer_params(m, n, key, scale=1e-2):
    """A helper function to randomly initialize weights and biases of a dense layer""" 
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k, scale) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

init_params = init_network_params(LAYER_SIZES, random.PRNGKey(0), scale=PARAM_SCALE)

In [ ]:
sharded_params = []
for i,(w,b) in enumerate(init_params):
  print(i, w.shape, b.shape)
  if i==0:
    w = jax.device_put(w, sharding.replicate(0))
    b = jax.device_put(b, sharding.replicate(0))
  elif i==1:
    w = jax.device_put(w, sharding.replicate(0))
    b = jax.device_put(b, sharding.replicate(0))
  elif i==2:
    w = jax.device_put(w, sharding.replicate())
    b = jax.device_put(b, sharding.replicate())
  sharded_params.append((w,b))


0 (10000, 784) (10000,)
1 (10000, 10000) (10000,)
2 (10, 10000) (10,)


In [ ]:
for (w,b) in init_params:
  jax.debug.visualize_array_sharding(w)
  jax.debug.visualize_array_sharding(b)

┌───────┐
│       │
│       │
│       │
│       │
│ TPU 0 │
│       │
│       │
│       │
│       │
└───────┘

┌───────┐
│ TPU 0 │
└───────┘

┌───────────────────────┐
│                       │
│                       │
│                       │
│                       │
│         TPU 0         │
│                       │
│                       │
│                       │
│                       │
└───────────────────────┘

┌───────┐
│ TPU 0 │
└───────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                    TPU 0                                     │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────┐
│ TPU 0 │
└───────┘

In [ ]:
for (w,b) in init_params:
  jax.debug.visualize_array_sharding(jax.device_put(w, sharding.replicate()))
  jax.debug.visualize_array_sharding(jax.device_put(b, sharding.replicate()))

┌───────────────────┐
│                   │
│                   │
│                   │
│                   │
│TPU 0,1,2,3,4,5,6,7│
│                   │
│                   │
│                   │
│                   │
└───────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

┌───────────────────────┐
│                       │
│                       │
│                       │
│                       │
│  TPU 0,1,2,3,4,5,6,7  │
│                       │
│                       │
│                       │
│                       │
└───────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                             TPU 0,1,2,3,4,5,6,7                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

In [ ]:
for (w,b) in sharded_params:
  jax.debug.visualize_array_sharding(w)
  jax.debug.visualize_array_sharding(b)

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

┌───────────┬───────────┐
│TPU 0,2,4,6│TPU 1,3,5,7│
└───────────┴───────────┘

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

┌───────────┬───────────┐
│TPU 0,2,4,6│TPU 1,3,5,7│
└───────────┴───────────┘

┌──────────────────────────────────────────────────────────────────────────────┐
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                             TPU 0,1,2,3,4,5,6,7                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

┌───────────────────┐
│TPU 0,1,2,3,4,5,6,7│
└───────────────────┘

In [ ]:
import time

params = sharded_params
for epoch in range(NUM_EPOCHS):
  start_time = time.time()
  losses = []
  for x, y in train_data:
    x = jnp.reshape(x, (len(x), NUM_PIXELS))
    y = one_hot(y, NUM_LABELS)
    x = jax.device_put(x, sharding.replicate(1))
    y = jax.device_put(y, sharding.replicate(1))
    #params = jax.device_put(params, sharding.replicate())
    params, loss_value = update(params, x, y, epoch)
    losses.append(jnp.sum(loss_value))
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_data)
  test_acc = accuracy(params, test_data)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set loss {}".format(jnp.mean(jnp.array(losses))))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

  jax.debug.visualize_array_sharding(params[0][0])

Epoch 0 in 6.76 sec
Training set loss 0.6637930870056152
Training set accuracy 0.8757923245429993
Test set accuracy 0.8863281607627869


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 1 in 4.04 sec
Training set loss 0.6224856376647949
Training set accuracy 0.890198290348053
Test set accuracy 0.8955078125


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 2 in 4.01 sec
Training set loss 0.6166571974754333
Training set accuracy 0.8960161209106445
Test set accuracy 0.900195300579071


...

┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 18 in 4.00 sec
Training set loss 0.58741295337677
Training set accuracy 0.9402149319648743
Test set accuracy 0.9404296875


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

Epoch 19 in 3.99 sec
Training set loss 0.5864367485046387
Training set accuracy 0.9415946006774902
Test set accuracy 0.941699206829071


┌───────────┬───────────┐
│           │           │
│           │           │
│           │           │
│           │           │
│TPU 0,2,4,6│TPU 1,3,5,7│
│           │           │
│           │           │
│           │           │
│           │           │
└───────────┴───────────┘

**!!! Do not forget to shutdown your Cloud TPU, or you'll spend much money on it!!!**